In [ ]:
# 安装固定版本的transformers库（重要：XTTS v2不支持>=4.50.0版本）
!pip install "transformers<4.50.0"

# 安装TTS引擎和相关依赖
!pip install --upgrade TTS

# 安装Web框架和API服务
!pip install fastapi uvicorn

# 安装网络代理工具
!pip install pyngrok

# 安装深度学习框架
!pip install torch

# 安装音频处理库
!pip install pydub

# 安装FFmpeg（音频格式转换必需）
!apt-get update && apt-get install -y ffmpeg


In [ ]:
# 导入必要的库
import torch
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import XttsAudioConfig, XttsArgs
from TTS.config.shared_configs import BaseDatasetConfig

# 将XTTS相关配置类添加到PyTorch安全加载白名单
# 这是为了解决PyTorch 2.6+版本的序列化安全限制
torch.serialization.add_safe_globals([
    XttsConfig,           # XTTS模型配置
    XttsAudioConfig,      # XTTS音频配置
    BaseDatasetConfig,    # 基础数据集配置
    XttsArgs             # XTTS参数配置
])

print("✅ PyTorch兼容性修复完成")
print(f"当前PyTorch版本: {torch.__version__}")


In [ ]:
# 配置ngrok认证token
# 请将下面的YOUR_NGROK_TOKEN替换为您在ngrok.com获取的实际token
NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # 🔑 请在这里填入您的ngrok token

# 验证token格式（基本检查）
if NGROK_TOKEN == "YOUR_NGROK_TOKEN" or len(NGROK_TOKEN) < 20:
    print("❌ 请先设置有效的ngrok token!")
    print("📖 获取步骤：")
    print("   1. 访问 https://ngrok.com 注册账户")
    print("   2. 登录后在Dashboard页面找到Authtoken")
    print("   3. 复制token并替换上面的YOUR_NGROK_TOKEN")
else:
    # 配置ngrok认证
    !ngrok config add-authtoken {NGROK_TOKEN}
    print("✅ ngrok认证配置成功")


In [ ]:
# 上传土耳其语音频样本
from google.colab import files
import os

print("📤 请上传您的土耳其语音频样本文件...")
print("💡 建议文件名：turkish_speaker.wav")
print("⏱️  建议时长：5-15秒")
print("🎯 要求：清晰的土耳其语发音，无背景噪音")

# 文件上传界面
uploaded = files.upload()

# 获取上传的文件名
if uploaded:
    speaker_filename = list(uploaded.keys())[0]
    print(f"✅ 音频样本上传成功: {speaker_filename}")
    print(f"📁 文件大小: {len(uploaded[speaker_filename]) / 1024:.1f} KB")
    
    # 检查文件是否存在
    if os.path.exists(speaker_filename):
        print(f"✅ 文件验证成功: {speaker_filename}")
        SPEAKER_WAV_PATH = speaker_filename
    else:
        print("❌ 文件上传失败，请重试")
else:
    print("❌ 未检测到上传文件")
    # 使用默认样本（如果有的话）
    SPEAKER_WAV_PATH = None


In [ ]:
# 导入TTS相关库
from TTS.api import TTS
import torch

# 检查GPU可用性
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🖥️  计算设备: {device}")

if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🎮 GPU型号: {gpu_name}")
    print(f"💾 GPU内存: {gpu_memory:.1f} GB")
else:
    print("⚠️  警告：未检测到GPU，将使用CPU（速度较慢）")

print("\n📥 正在下载XTTS v2模型...")
print("⏱️  首次下载需要几分钟时间，请耐心等待")
print("📦 模型大小约1.8GB")


In [ ]:
# 初始化XTTS v2模型
try:
    # 创建TTS实例
    # model_name: 指定使用XTTS v2多语言模型
    # progress_bar: 是否显示进度条
    # gpu: 是否使用GPU加速
    tts = TTS(
        model_name="tts_models/multilingual/multi-dataset/xtts_v2",
        progress_bar=True,
        gpu=(device == "cuda")
    )
    
    print("✅ XTTS v2模型加载成功！")
    print(f"🌍 支持的语言: {', '.join(tts.languages) if hasattr(tts, 'languages') else '多种语言'}")
    
except Exception as e:
    print(f"❌ 模型加载失败: {e}")
    print("🔧 可能的解决方案：")
    print("   1. 检查网络连接")
    print("   2. 重启Colab运行时")
    print("   3. 确认transformers版本正确")
    raise


In [ ]:
# 导入FastAPI相关库
from fastapi import FastAPI, Request, HTTPException
from fastapi.responses import FileResponse, JSONResponse
from pydub import AudioSegment
import uvicorn
import threading
import os
import traceback
from datetime import datetime

# 创建FastAPI应用实例
app = FastAPI(
    title="Telegram Turkish TTS API",
    description="将土耳其语文本转换为MP3语音的API服务",
    version="1.0.0"
)

# 全局变量：保存TTS实例和音频样本路径
TTS_MODEL = tts
SPEAKER_SAMPLE = SPEAKER_WAV_PATH

print("🚀 FastAPI应用创建成功")
print(f"📁 使用的音频样本: {SPEAKER_SAMPLE}")


In [ ]:
# 定义API路由

@app.get("/")
async def root():
    """根路径：API状态检查"""
    return {
        "message": "Telegram Turkish TTS API",
        "status": "running",
        "timestamp": datetime.now().isoformat(),
        "version": "1.0.0",
        "endpoints": {
            "tts": "POST /tts - 文本转语音",
            "health": "GET /health - 健康检查"
        }
    }

@app.get("/health")
async def health_check():
    """健康检查端点"""
    try:
        # 检查TTS模型是否可用
        model_status = "available" if TTS_MODEL else "unavailable"
        speaker_status = "available" if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE) else "unavailable"
        
        return {
            "status": "healthy",
            "tts_model": model_status,
            "speaker_sample": speaker_status,
            "timestamp": datetime.now().isoformat()
        }
    except Exception as e:
        return JSONResponse(
            status_code=500,
            content={"status": "unhealthy", "error": str(e)}
        )

@app.post("/tts")
async def create_tts(request: Request):
    """
    文本转语音API端点
    
    接收JSON格式的文本，返回MP3格式的语音文件
    """
    try:
        # 解析请求数据
        data = await request.json()
        text = data.get("text", "")
        language = data.get("language", "tr")  # 默认土耳其语
        
        # 输入验证
        if not text.strip():
            raise HTTPException(status_code=400, detail="文本内容不能为空")
        
        if len(text) > 1000:
            raise HTTPException(status_code=400, detail="文本长度不能超过1000字符")
        
        print(f"🎯 收到TTS请求: {text[:50]}...") if len(text) > 50 else print(f"🎯 收到TTS请求: {text}")
        print(f"🌍 语言: {language}")
        
        # 生成唯一的输出文件名
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        wav_filename = f"output_{timestamp}.wav"
        mp3_filename = f"output_{timestamp}.mp3"
        
        # 生成语音
        if SPEAKER_SAMPLE and os.path.exists(SPEAKER_SAMPLE):
            # 使用声音克隆
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                speaker_wav=SPEAKER_SAMPLE,
                language=language
            )
            print("✅ 语音生成完成（使用声音克隆）")
        else:
            # 使用默认说话人
            TTS_MODEL.tts_to_file(
                text=text,
                file_path=wav_filename,
                language=language
            )
            print("✅ 语音生成完成（使用默认说话人）")
        
        # 转换为MP3格式
        if os.path.exists(wav_filename):
            sound = AudioSegment.from_wav(wav_filename)
            sound.export(mp3_filename, format="mp3")
            
            # 清理临时WAV文件
            os.remove(wav_filename)
            
            print(f"🎵 MP3文件生成成功: {mp3_filename}")
            
            # 返回MP3文件
            return FileResponse(
                mp3_filename,
                media_type="audio/mpeg",
                filename=f"tts_{timestamp}.mp3"
            )
        else:
            raise HTTPException(status_code=500, detail="语音文件生成失败")
            
    except HTTPException:
        # 重新抛出HTTP异常
        raise
    except Exception as e:
        # 捕获其他异常
        error_msg = f"TTS处理错误: {str(e)}"
        print(f"❌ {error_msg}")
        print(f"🔍 详细错误: {traceback.format_exc()}")
        raise HTTPException(status_code=500, detail=error_msg)

print("✅ API路由定义完成")


In [ ]:
# 启动API服务器和ngrok隧道
from pyngrok import ngrok
import time

# 定义API服务器运行函数
def run_api_server():
    """在独立线程中运行FastAPI服务器"""
    try:
        # 使用uvicorn运行FastAPI应用
        # host="0.0.0.0": 监听所有网络接口
        # port=7860: 使用端口7860（Hugging Face Spaces常用端口）
        # log_level="info": 设置日志级别
        uvicorn.run(
            app,
            host="0.0.0.0",
            port=7860,
            log_level="info"
        )
    except Exception as e:
        print(f"❌ API服务器启动失败: {e}")

# 在后台线程启动API服务器
print("🚀 正在启动FastAPI服务器...")
print("📡 服务器地址: http://localhost:7860")
print("⏱️  服务器启动需要几秒钟时间")

# 创建并启动服务器线程
server_thread = threading.Thread(target=run_api_server)
server_thread.daemon = True  # 设置为守护线程
server_thread.start()

# 等待服务器启动
time.sleep(5)

try:
    print("🌐 正在创建ngrok隧道...")
    
    # 创建HTTP隧道到本地7860端口
    public_tunnel = ngrok.connect(7860)
    public_url = public_tunnel.public_url
    
    print("\n✅ 系统启动成功！")
    print("="*60)
    print(f"🌍 公网访问地址: {public_url}")
    print(f"📋 API文档地址: {public_url}/docs")
    print(f"❤️  健康检查: {public_url}/health")
    print(f"🎵 TTS端点: {public_url}/tts")
    print("="*60)
    
    # 保存公网URL供后续使用
    PUBLIC_API_URL = public_url
    
except Exception as e:
    print(f"❌ ngrok隧道创建失败: {e}")
    print("🔧 可能的解决方案：")
    print("   1. 检查ngrok token是否正确")
    print("   2. 确认FastAPI服务器已启动")
    print("   3. 重启Colab运行时")
    PUBLIC_API_URL = None


In [ ]:
# 生成Telegram Bot配置信息和n8n工作流
import json

print("🤖 Telegram Bot 配置指南")
print("="*50)
print()
print("📋 第一步：创建Telegram Bot")
print("   1. 在Telegram中搜索 @BotFather")
print("   2. 发送 /newbot 命令")
print("   3. 按提示设置Bot名称和用户名")
print("   4. 获取Bot Token（格式：123456789:ABCDEF...）")
print()
print("📋 第二步：配置Bot权限")
print("   1. 发送 /setprivacy 给 @BotFather")
print("   2. 选择您的Bot")
print("   3. 选择 'Disable' 以接收所有消息")
print()
print("📋 第三步：记录必要信息")
print("   🔑 Bot Token: [从BotFather获取]")
print("   👤 Chat ID: [与Bot对话后从n8n webhook获取]")
print()

if 'PUBLIC_API_URL' in locals() and PUBLIC_API_URL:
    print("📋 第四步：API信息（用于n8n配置）")
    print(f"   🌍 TTS API URL: {PUBLIC_API_URL}/tts")
    print("   📝 请求方法: POST")
    print("   📋 请求格式: JSON")
    print("   📄 请求体示例:")
    print("   {")
    print('     "text": "要转换的土耳其语文本",') 
    print('     "language": "tr"')
    print("   }")
    print("   📁 响应: MP3音频文件")
else:
    print("❌ API URL未生成，请先完成前面的步骤")

print()
print("📋 第五步：n8n工作流配置")
print("   1. 在n8n中创建新工作流")
print("   2. 添加Webhook节点（接收Telegram消息）")
print("   3. 添加IF节点（检查是否有文本消息）")
print("   4. 添加HTTP请求节点（调用TTS API）")
print("   5. 添加Telegram节点（发送语音消息）")
print("   6. 添加响应Webhook节点（确认处理完成）")
print()
print("💡 详细配置说明：")
print("   🔧 Webhook节点：路径设为 'telegram-webhook'")
print("   🔧 HTTP请求节点：设置响应格式为 '文件'")
print("   🔧 Telegram节点：操作选择 '发送语音'")
print()
print("🎯 测试步骤：")
print("   1. 激活n8n工作流")
print("   2. 配置Telegram webhook指向n8n")
print("   3. 向Bot发送文本消息")
print("   4. 检查是否收到语音回复")


In [ ]:
# 测试TTS功能
import IPython.display as ipd
import requests

# 测试文本（土耳其语）
test_text = "Merhaba! Bu bir test mesajıdır. Türkçe metin okuma testi."
print(f"🧪 测试文本: {test_text}")

try:
    # 生成测试语音
    print("\n🎵 正在生成测试语音...")
    
    if 'SPEAKER_WAV_PATH' in locals() and SPEAKER_WAV_PATH and os.path.exists(SPEAKER_WAV_PATH):
        # 使用声音克隆
        tts.tts_to_file(
            text=test_text,
            file_path="test_output.wav",
            speaker_wav=SPEAKER_WAV_PATH,
            language="tr"  # 土耳其语
        )
        print("✅ 测试语音生成成功（使用声音克隆）")
    else:
        # 使用默认说话人
        tts.tts_to_file(
            text=test_text,
            file_path="test_output.wav",
            language="tr"  # 土耳其语
        )
        print("✅ 测试语音生成成功（使用默认说话人）")
    
    # 播放生成的语音
    print("\n🎧 生成的语音预览：")
    display(ipd.Audio("test_output.wav"))
    
except Exception as e:
    print(f"❌ 测试失败: {e}")
    print("💡 请检查模型是否正确加载")

# API测试
if 'PUBLIC_API_URL' in locals() and PUBLIC_API_URL:
    print("\n🌐 测试API接口...")
    
    test_payload = {
        "text": "API test mesajı. Ses kalitesi nasıl?",
        "language": "tr"
    }
    
    try:
        print(f"📤 发送请求: {test_payload['text']}")
        
        response = requests.post(
            f"{PUBLIC_API_URL}/tts",
            json=test_payload,
            timeout=30
        )
        
        if response.status_code == 200:
            # 保存返回的MP3文件
            api_test_filename = "api_test_output.mp3"
            with open(api_test_filename, "wb") as f:
                f.write(response.content)
            
            print("✅ API测试成功！")
            print(f"📁 生成文件: {api_test_filename}")
            print(f"📊 文件大小: {len(response.content) / 1024:.1f} KB")
            
            # 播放生成的音频
            print("\n🎧 API生成的语音：")
            display(ipd.Audio(api_test_filename))
            
        else:
            print(f"❌ API测试失败，状态码: {response.status_code}")
            print(f"📄 错误信息: {response.text}")
            
    except requests.exceptions.Timeout:
        print("⏰ API请求超时（这是正常的，TTS生成需要时间）")
        print("💡 请稍后手动测试API")
    except Exception as api_error:
        print(f"❌ API测试异常: {api_error}")
else:
    print("❌ 无法测试API，因为公网URL未生成")
